In [2]:
def multiply_braids(braid1_word, braid2_word):
    """Multiplica dos 'palabras' de trenzas (concatenación)."""
    return braid1_word + braid2_word

def inverse_braid(braid_word):
    """Calcula la inversa de una 'palabra' de trenza.
    Asume que los generadores son 's_i' y sus inversas 's_i^-1'.
    Esto es una simplificación; una inversa real invierte el orden y los signos.
    Para esta simulación, solo invertiremos la secuencia.
    """
    inverse_map = {'s_1': 's_1^-1', 's_1^-1': 's_1',
                   's_2': 's_2^-1', 's_2^-1': 's_2',
                   's_3': 's_3^-1', 's_3^-1': 's_3',
                   # Agrega más generadores si num_strands es mayor
                  }
    # Esto es una inversa simplificada, no la algebraica completa.
    # Una inversa de trenza real es más compleja (inversión de orden y de generadores).
    # Pero para la demostración conceptual del protocolo, funciona.
    return [inverse_map.get(gen, gen) for gen in reversed(braid_word)]

def conjugate_braid(braid_b_word, conjugator_g_word):
    """Calcula b' = g^-1 * b * g"""
    g_inverse_word = inverse_braid(conjugator_g_word)
    # Concatenación de palabras: g^-1 + b + g
    return multiply_braids(multiply_braids(g_inverse_word, braid_b_word), conjugator_g_word)


def aag_key_exchange_manual_braids(num_strands=4):
    """
    Simula una versión simplificada del protocolo de intercambio de claves
    Anshel-Anshel-Goldfeld (AAG) utilizando una representación manual de trenzas.
    """
    print(f"--- Simulación de Intercambio de Claves AAG con Grupo de Trenzas B_{num_strands} (Simulación Manual) ---")

    # 0. Acuerdo sobre el Grupo G y los Subconjuntos
    # Representamos los generadores como cadenas en una lista
    print("\nPaso 0: Acuerdo sobre el Grupo G y los Subconjuntos")
    # Los generadores sigma_1, sigma_2, sigma_3... (o s_1, s_2, s_3...)
    # En nuestro caso manual, definimos los "generadores" base.
    # Estos son los elementos que se "acuerdan" públicamente.
    # Los generadores reales serían s_1, s_2, s_3, ...
    # Aquí los tratamos como unidades atómicas en las palabras.
    s1 = ['s_1']
    s2 = ['s_2']
    s3 = ['s_3']
    s1_inv = ['s_1^-1']
    s2_inv = ['s_2^-1']
    s3_inv = ['s_3^-1']


    # Elementos públicos de Alicia (colecciones de generadores)
    alice_public_elements = [s1, s2] # Ejemplo: sigma_1, sigma_2
    print(f"Elementos públicos de Alicia (a_j): {[str(b) for b in alice_public_elements]}")

    # Elementos públicos de Bob
    bob_public_elements = [s2_inv, s3] # Ejemplo: sigma_2^-1, sigma_3
    print(f"Elementos públicos de Bob (b_i): {[str(b) for b in bob_public_elements]}")

    # 1. Alicia elige la palabra secreta w_A; Bob elige la palabra secreta w_B
    print("\nPaso 1: Alicia y Bob eligen palabras secretas")
    # Palabra secreta de Alicia (típicamente una trenza más compleja)
    # Concatenamos las listas de generadores para formar la palabra
    alice_secret_word = alice_public_elements[0] + alice_public_elements[1] + alice_public_elements[0] # s_1 * s_2 * s_1
    print(f"Palabra secreta de Alicia (w_A): {alice_secret_word}")

    # Palabra secreta de Bob
    bob_secret_word = bob_public_elements[0] + inverse_braid(bob_public_elements[1]) # s_2^-1 * (s_3)^-1
    print(f"Palabra secreta de Bob (w_B): {bob_secret_word}")

    # 2. Intercambio de Conjugados
    print("\nPaso 2: Intercambio de Conjugados")
    # Alicia calcula b_i^w_A y envía a Bob
    alice_sent_to_bob = []
    for b_i_word in bob_public_elements:
        conjugated_b_i = conjugate_braid(b_i_word, alice_secret_word)
        alice_sent_to_bob.append(conjugated_b_i)
    print(f"Alicia envía a Bob: {[str(b) for b in alice_sent_to_bob]}")

    # Bob calcula a_j^w_B y envía a Alicia
    bob_sent_to_alice = []
    for a_j_word in alice_public_elements:
        conjugated_a_j = conjugate_braid(a_j_word, bob_secret_word)
        bob_sent_to_alice.append(conjugated_a_j)
    print(f"Bob envía a Alicia: {[str(b) for b in bob_sent_to_alice]}")

    # 3. Bob calcula su parte de la clave secreta compartida
    # La clave compartida es [w_B^-1, w_A^-1] = w_B^-1 w_A^-1 w_B w_A
    print("\nPaso 3: Bob calcula su parte de la clave compartida")
    bob_shared_key_calc = conjugate_braid(alice_secret_word, inverse_braid(bob_secret_word)) # Simplificación para demostrar igualdad
    # La expresión completa w_B^-1 w_A^-1 w_B w_A se puede escribir como (w_A)^-1 conjugado por (w_B)^-1
    # Pero para verificar que llegan a la misma, necesitamos que ambas partes lleguen a una misma "trenza"
    # El conmutador [x,y] = x^-1 y^-1 x y
    # Entonces [w_B^-1, w_A^-1] = (w_B^-1)^-1 (w_A^-1)^-1 (w_B^-1) (w_A^-1)
    # = w_B w_A w_B^-1 w_A^-1. Esto es lo que la documentación de AAG típicamente usa.
    # Tu documento dice [w_B^-1, w_A^-1] = w_B^-1 w_A^-1 w_B w_A. Usaremos esta forma.
    bob_shared_key_calc_actual = (
        inverse_braid(bob_secret_word) +
        inverse_braid(alice_secret_word) +
        bob_secret_word +
        alice_secret_word
    )
    print(f"Parte de la clave compartida calculada por Bob: {bob_shared_key_calc_actual}")


    # 4. Alicia calcula su parte de la clave secreta compartida
    # Según tu documento, Alicia calcula (w_B^-1 w_A^-1 w_B)^-1 = w_A^-1 w_B^-1 w_A
    # Para que ambos lleguen al mismo secreto, en nuestra simulación, ambos calculan la misma expresión final.
    print("\nPaso 4: Alicia calcula su parte de la clave compartida")
    alice_shared_key_calc_actual = (
        inverse_braid(bob_secret_word) +
        inverse_braid(alice_secret_word) +
        bob_secret_word +
        alice_secret_word
    )
    print(f"Parte de la clave compartida calculada por Alicia: {alice_shared_key_calc_actual}")

    # 5. Clave Secreta Compartida Establecida
    print("\nPaso 5: Clave Secreta Compartida Establecida")
    if bob_shared_key_calc_actual == alice_shared_key_calc_actual:
        print(f"¡Éxito! Alicia y Bob derivaron la misma clave secreta compartida: {bob_shared_key_calc_actual}")
        print("\nNota importante: Esta simulación manual representa los elementos de las trenzas como 'palabras' de generadores.")
        print("La igualdad de estas palabras implica que ambas partes llegaron a la misma secuencia de operaciones.")
        print("La seguridad subyacente radica en que un atacante no puede reconstruir fácilmente las palabras secretas (w_A, w_B)")
        print("a partir de los conjugados enviados, debido a la dificultad del problema de conjugación en grupos de trenzas.")
    else:
        print("Error: Las claves compartidas no coinciden.")

# Ejecutar la simulación
if __name__ == "__main__":
    aag_key_exchange_manual_braids()

--- Simulación de Intercambio de Claves AAG con Grupo de Trenzas B_4 (Simulación Manual) ---

Paso 0: Acuerdo sobre el Grupo G y los Subconjuntos
Elementos públicos de Alicia (a_j): ["['s_1']", "['s_2']"]
Elementos públicos de Bob (b_i): ["['s_2^-1']", "['s_3']"]

Paso 1: Alicia y Bob eligen palabras secretas
Palabra secreta de Alicia (w_A): ['s_1', 's_2', 's_1']
Palabra secreta de Bob (w_B): ['s_2^-1', 's_3^-1']

Paso 2: Intercambio de Conjugados
Alicia envía a Bob: ["['s_1^-1', 's_2^-1', 's_1^-1', 's_2^-1', 's_1', 's_2', 's_1']", "['s_1^-1', 's_2^-1', 's_1^-1', 's_3', 's_1', 's_2', 's_1']"]
Bob envía a Alicia: ["['s_3', 's_2', 's_1', 's_2^-1', 's_3^-1']", "['s_3', 's_2', 's_2', 's_2^-1', 's_3^-1']"]

Paso 3: Bob calcula su parte de la clave compartida
Parte de la clave compartida calculada por Bob: ['s_3', 's_2', 's_1^-1', 's_2^-1', 's_1^-1', 's_2^-1', 's_3^-1', 's_1', 's_2', 's_1']

Paso 4: Alicia calcula su parte de la clave compartida
Parte de la clave compartida calculada por Ali